# Load project YAML from - Git, Zip, Tar source

After you create your project YAML you can simply load that project and run, build and deploy your function .
You can found an addional information of how to create a project YAML for CI/CD [here](https://github.com/GiladShapira94/load-project-example/blob/master/CI-CD%20automation%20creation.md)

In this session you would learn how to:
* Load or Create a new project for remote URL
* get a function object 
* run a function
* build a function
* deploy a function
* run a workflow

Load a Project from exicting project YAML, you can clone you entire repository from git and use the project YAML in your repository using the url parameter

Install mlrun - if does not install use ``pip install mlrun==<mlrun server version>`` or ``sh align_mlrun.sh`` (our default mlrun installer - automatically install the server version)

In [1]:
import mlrun

#### Load or Create a new project
For loading a project you can use :
1. [load_project](https://docs.mlrun.org/en/latest/api/mlrun.projects.html?highlight=load_project#mlrun.projects.load_project) thats Load an MLRun project from git or tar or dir 
````
# load the project and run the 'main' workflow
project = load_project("./", "git://github.com/mlrun/project-demo.git",url=<name (in DB) or git or tar.gz or .zip sources archive path>)
````
2. [new_project](https://docs.mlrun.org/en/latest/api/mlrun.projects.html?highlight=new_project#mlrun.projects.new_project) thats Create a new MLRun project, optionally load it from a yaml/zip/git template.
````
# create a project with local and marketplace functions, a workflow, and an artifact
project = mlrun.new_project("myproj", "./", init_git=True, description="my new project",url=<name (in DB) or git or tar.gz or .zip sources archive path>)
````
***Important Note-*** If you want to run your code locally you must clone the files to the context directory, execute automaticlly when you load project or create new project and define a url params


In [2]:
project = mlrun.load_project('./')

In [3]:
print(project.to_yaml())

kind: project
metadata:
  name: new-git-proj
spec:
  functions:
  - name: nuclio
    kind: nuclio
    image: mlrun/mlrun
    handler: multi:multi_3
    with_repo: true
  - name: my_job
    kind: job
    image: mlrun/mlrun
    handler: my_job.func
    with_repo: true
  - name: my_job_with_requirements
    kind: job
    image: mlrun/mlrun
    handler: my_job_with_requirements.log_df
    with_repo: true
    requirements:
    - pandas
  - url: serving.yaml
    name: serving
    with_repo: true
  workflows: []
  artifacts: []
  source: git://github.com/GiladShapira94/load-project-example.git#refs/heads/master
  origin_url: git://github.com/GiladShapira94/load-project-example.git#refs/heads/master
  load_source_on_run: true
  desired_state: online



The source define your source code this source need to include all your project files  - MLRun knows to point to the files in the source address and run the functions based on the source files

In [4]:
source = 'git://github.com/GiladShapira94/load-project-example.git#master'

The pull_at_runtime flag will determine if the code is loaded in runtime or added to the image during build. the first (at runtime) option is better for debugging while the secound is better for production. Note that if you choose the 2nd option you'll need to build the function before run.

In [5]:
project.set_source(source=source,pull_at_runtime=True)

#### Run Function 
For run a function you will need to use the [run_function](https://docs.mlrun.org/en/latest/api/mlrun.projects.html?highlight=run_function#mlrun.projects.MlrunProject.run_function) method.
This method allows you to run a MLRun **jobs** locally and remotely as long as there is no requirments ( if there is any requirments you will need to build a new image before you run a function)
It is equivalent to func.run method.
````
project.run_function(function='<function name>',params={'num':3},local=False)
````
````
project.run_function(function='<function name>',params={'num':3},local=True)
````

In [6]:
project.run_function(function='my_job',params={'num':3})

> 2022-08-03 11:16:34,492 [info] starting run my_job-func uid=7c71dbbd4c2a4fa880f97eb3fc79bdfd DB=http://mlrun-api:8080
> 2022-08-03 11:16:34,777 [info] Job is running in the background, pod: my-job-func-q8rg7
> 2022-08-03 11:16:38,911 [info] extracting source from git://github.com/GiladShapira94/load-project-example.git#master to /mlrun/code
24
> 2022-08-03 11:16:39,067 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
new-git-proj,...fc79bdfd,0,Aug 03 11:16:38,completed,my_job-func,v3io_user=shapirakind=jobowner=shapiramlrun/client_version=1.1.0-rc12host=my-job-func-q8rg7,,num=3,num=24return=24,


> 2022-08-03 11:16:41,307 [info] run executed, status=completed


**Changed pull_at_runtime to False for pull files on image creation**

In [7]:
project.set_source(source=source)
print(project.to_yaml())

kind: project
metadata:
  name: new-git-proj
spec:
  functions:
  - name: nuclio
    kind: nuclio
    image: mlrun/mlrun
    handler: multi:multi_3
    with_repo: true
  - name: my_job
    kind: job
    image: mlrun/mlrun
    handler: my_job.func
    with_repo: true
  - name: my_job_with_requirements
    kind: job
    image: mlrun/mlrun
    handler: my_job_with_requirements.log_df
    with_repo: true
    requirements:
    - pandas
  - url: serving.yaml
    name: serving
    with_repo: true
  workflows: []
  artifacts: []
  source: git://github.com/GiladShapira94/load-project-example.git#master
  origin_url: git://github.com/GiladShapira94/load-project-example.git#refs/heads/master
  load_source_on_run: false
  desired_state: online



#### Deploy Function
For deplying remote function as nuclio or serving you will need to use the [deploy_function](https://docs.mlrun.org/en/latest/api/mlrun.projects.html?highlight=deploy_function#mlrun.projects.MlrunProject.deploy_function) method.
You must use this method before invoke nuclio or serving fucntions.
It is equivalent to func.deploy() method.
````
nuclio_func=project.deploy_function(function='<function name>')

nuclio_func.function.invoke('/',{'int':4})
````

In [8]:
serving_func=project.deploy_function('serving')

> 2022-08-03 11:16:41,476 [info] Starting remote function deploy
2022-08-03 11:16:41  (info) Deploying function
2022-08-03 11:16:41  (info) Building
2022-08-03 11:16:41  (info) Staging files and preparing base images
2022-08-03 11:16:42  (info) Building processor image
2022-08-03 11:18:27  (info) Build complete
2022-08-03 11:18:40  (info) Function deploy complete
> 2022-08-03 11:18:41,194 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-new-git-proj-serving.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['new-git-proj-serving-new-git-proj.default-tenant.app.cust-cs-il-3-4-1.iguazio-cd1.com/']}


In [9]:
serving_func.function.invoke('/',5)

> 2022-08-03 11:18:41,286 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-new-git-proj-serving.default-tenant.svc.cluster.local:8080/'}


13

In [10]:
# project.spec.origin_url='git://github.com/GiladShapira94/load-project-example.git#refs/heads/master'
nuclio_func=project.deploy_function(function='nuclio')

> 2022-08-03 11:18:41,328 [info] Starting remote function deploy
2022-08-03 11:18:41  (info) Deploying function
2022-08-03 11:18:41  (info) Building
2022-08-03 11:18:41  (info) Staging files and preparing base images
2022-08-03 11:18:41  (info) Building processor image
2022-08-03 11:20:16  (info) Build complete
2022-08-03 11:20:24  (info) Function deploy complete
> 2022-08-03 11:20:25,895 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-new-git-proj-nuclio.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['new-git-proj-nuclio-new-git-proj.default-tenant.app.cust-cs-il-3-4-1.iguazio-cd1.com/']}


In [11]:
nuclio_func.function.invoke('/',{'int':4})

> 2022-08-03 11:20:25,980 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-new-git-proj-nuclio.default-tenant.svc.cluster.local:8080/'}


{'int': 12}